In [426]:
import numpy as np
import time
import pandas as pd

from game_functions.environment import sbfeatures as sbf
from game_functions.model import brain

In [427]:
env = sbf.Environment()

init


In [428]:
env.controller(initialize=True, close=True)
env.controller(initialize=True, opens=True)
env.action(5, delay=0.1)

5

In [429]:
env.make()

launching game...


In [430]:
env.start_game(botsOn=True, mode='easy')

In [431]:
env.initalize_memory()

hwnd: 200002
pid: c_long(9788)


In [432]:
print(env.observation())
#env.__grab_addresses__()
#env.__grab_features__()

[100.0, 3.0, 4.0, 100.0, -8.0, -6.0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 3.0, -8.0, 4.0, -6.0]


In [433]:
print(env.p1_score, env.p2_score)

0 0


In [434]:
env.reward_structure(p1_death=-1, p2_death=1, p1_damage=-0.01, p2_damage=0.01)

Reward structure defined!


In [435]:
#print(env.features)
print(env.observation())

[100.0, 3.0, 4.3, 100.0, -8.0, -5.7, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0.0, 0.0, 0.2999999999999998, 0.2999999999999998]


In [436]:
print(env.reward(stale_observation=env.features, 
                 new_observation=env.observation()))

0


In [437]:

env.action(5, delay=0.1)

5

In [457]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import Adam
from keras.layers import Dropout
from keras.layers import advanced_activations
from keras import initializers

def keras_model(inputs, dropouts=0.05, opti=1e-6, clipping=None, clipval=None):
    keras_model = Sequential()
    
    keras_model.add(Dense(128, input_dim=inputs, 
                          kernel_initializer=initializers.RandomNormal(mean=1.0, stddev=0.1), 
                          activation='relu'))
    #keras_model.add(advanced_activations.LeakyReLU(alpha=0.01))
    #keras_model.add(advanced_activations.ELU(alpha=1))
    keras_model.add(Dropout(dropouts))
    
    keras_model.add(Dense(128, input_dim=inputs, 
                          kernel_initializer=initializers.RandomNormal(mean=1.0, stddev=0.1),
                          activation='relu'))
    #keras_model.add(advanced_activations.LeakyReLU(alpha=0.01))
    #keras_model.add(advanced_activations.ELU(alpha=1))
    keras_model.add(Dropout(dropouts))
    
    keras_model.add(Dense(1))
    adam = Adam(lr=opti, clipnorm=clipping, clipvalue=clipval)
    keras_model.compile(loss='mse', optimizer='adam')
    return keras_model

In [460]:
input_shape = 17


Qas = keras_model(inputs=input_shape, dropouts=0.1, opti=1e-6, clipping=None, clipval=None)

#Qas.load_weights('SB-Qas-v1-20170620.keras')
Qas.load_weights('model_weights//kill_rewards_TD25_LA//2750000_model_backup_20170627_2326.keras')
#Qas.load_weights('model_weights//kill_rewards_v2//1750000_model_backup_20170626_0543.keras')

In [461]:
import time
import random
import datetime

episode = 0
training_backup = 5000
save_location = "training_data\\"
training_batch = []

# action settings
randomness = True

#score keeping
p1_score = 0
p2_score = 0

#init#
prev_observation = env.observation()[:]
observation = env.observation()[:]
header = ['episode','p1h', 'p1_x',' p1_y', 'p2h', 'p2_x',' p2_y',
          'right_of', 'left_of', 'above', 'below', 'x_clear', 'y_clear',
          'cd1', 'cd2',
         'aim_down', 'aim_left', 'aim_up', 'aim_right',
         'p1_x_delta', 'p2_x_delta', 'p1_y_delta', 'p2_y_delta',
         'action', 'reward', 'p1_score', 'p2_score']

while True:
    try:
        episode += 1
        
        #random action
        if randomness:
            if random.randint(0,10) <= -1:
                action = random.randint(0, 7)

        env.action(action, delay=0.1)
        
        observation = env.observation()[:]
        reward = env.reward(stale_observation=prev_observation, 
                            new_observation=observation)
        prev_observation = observation
        
        datapoint = [episode]
        datapoint.extend(observation)
        datapoint.append(action)
        datapoint.append(reward)
        datapoint.extend([env.p1_score + 25000, env.p2_score + 25000])
        training_batch.append(datapoint)
        
        #-------model-----------#
        modelpoint = list(datapoint)
        modelpoint.pop(-1) # p2 score
        modelpoint.pop(-1) # p1  score
        modelpoint.pop(-1) # reward
        modelpoint.pop(-1) # action
        modelpoint.pop(-1) # delta
        modelpoint.pop(-1) # delta
        modelpoint.pop(-1) # delta
        modelpoint.pop(-1) # delta
        modelpoint.pop(4) # p2 health
        modelpoint.pop(1) # p1 health
        modelpoint.pop(0) # episode
        
        predictions = env.create_predictions(modelpoint)
        action_rewards = Qas.predict(predictions[1:-1])
        
        #---action = np.where(advantage == np.amax(advantage))[0][0]
        action = np.where(action_rewards == np.amax(action_rewards))[0][0] # greedy action
        
        #-----------backup----------#
        if (len(training_batch) % training_backup) == 0:
            name =  save_location + str(episode) + '_training_backup_' + datetime.datetime.today().strftime('%Y%m%d_%H%M') + '.csv'
            exp = pd.DataFrame(training_batch)
            exp.columns = header
            exp.to_csv(name, index=False)
            print('Saving Training Experience...')
            training_batch = []
        
    except Exception:
        episode -= 1
        print('Error Reading Memory')
        time.sleep(0.25)
        
; #4294942296

Error Reading Memory
Error Reading Memory
Error Reading Memory
Error Reading Memory
Error Reading Memory
Error Reading Memory
Saving Training Experience...
Error Reading Memory
Saving Training Experience...


KeyboardInterrupt: 

In [419]:
predictions[1:-1]

[[1.4, -7.0, -12.2, -7.2, 0, 1, 1, 0, 0, 1, 2, 0, 0, 0, 0, 1, 1],
 [1.4, -7.0, -12.2, -7.2, 0, 1, 1, 0, 0, 1, 2, 0, 0, 0, 0, 1, 2],
 [1.4, -7.0, -12.2, -7.2, 0, 1, 1, 0, 0, 1, 2, 0, 0, 0, 0, 1, 3],
 [1.4, -7.0, -12.2, -7.2, 0, 1, 1, 0, 0, 1, 2, 0, 0, 0, 0, 1, 4],
 [1.4, -7.0, -12.2, -7.2, 0, 1, 1, 0, 0, 1, 2, 0, 0, 0, 0, 1, 5],
 [1.4, -7.0, -12.2, -7.2, 0, 1, 1, 0, 0, 1, 2, 0, 0, 0, 0, 1, 6]]

In [40]:
modelpoint = []
modelpoint.extend(datapoint)
print(modelpoint)

[1, 100.0, 2.0, 6.2, 100.0, -7.0, -4.8, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0.0, 0.0, 0.0, 0.0, 5, 0, 0, 0]


In [41]:
modelpoint.pop(-1)
modelpoint.pop(-1)
modelpoint.pop(-1)
modelpoint.pop(-1)
modelpoint.pop(3)
modelpoint.pop(0)

print(modelpoint)
print(len(modelpoint))

[100.0, 2.0, 100.0, -7.0, -4.8, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0.0, 0.0, 0.0, 0.0]
21


In [13]:
env.switch = True

In [14]:
env.observation()[:]

[100.0,
 -9.3,
 -7.0,
 34.0,
 2.9,
 -7.1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0.0,
 0.0,
 0.0,
 0.0]

In [14]:
env.switch

True

In [12]:
env.p2_dead

AttributeError: 'Environment' object has no attribute 'p2_dead'

In [13]:
env.p2_score

0

In [14]:
env.p2_score_prev

1

In [14]:
env.__grab_features__()

p2_dead
player 2 dead?
apparently
100.0 -4.0


[100.0,
 -1.768,
 -6.64,
 -4.0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0.0,
 0,
 0.0,
 0]